___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Question and Answer Chat Bots

----

------

## Loading the Data

We will be working with the Babi Data Set from Facebook Research.

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698


In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("/content/drive/MyDrive/train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [ ]:
with open("/content/drive/MyDrive/test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

In [ ]:
#y_train = []
#for story, question, answer in train_data:
#    y_train.append(answer)
#y_train_np = np.array(y_train)

In [ ]:
#train_data[0][2]

In [ ]:
train_len_list = [(len(obj[0]) + len(obj[1])) for obj in train_data]

In [ ]:
max(train_len_list)

162

----

In [ ]:
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

In [ ]:
df_train

,0,1,2
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no
1,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bathroom, ?]",no
2,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, office, ?]",no
3,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
4,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
...,...,...,...
9995,"[Daniel, went, to, the, office, ., Sandra, jou...","[Is, Sandra, in, the, kitchen, ?]",no
9996,"[Daniel, went, to, the, office, ., Sandra, jou...","[Is, Daniel, in, the, bedroom, ?]",yes
9997,"[Daniel, went, to, the, office, ., Sandra, jou...","[Is, Sandra, in, the, hallway, ?]",no
9998,"[Daniel, went, to, the, office, ., Sandra, jou...","[Is, Mary, in, the, kitchen, ?]",no


In [ ]:
## Change columns name ###
df_train = df_train.rename(columns={0: "Text", 1: "Question",2:"Answer"})
df_test = df_test.rename(columns={0: "Text", 1: "Question",2:"Answer"})

In [ ]:
## convert list to sentence by ' '.join and apply to first two columns##
df_train["Text"] = df_train["Text"].apply(lambda x: ' '.join(x))
df_train["Question"] = df_train["Question"].apply(lambda x: ' '.join(x))
df_test["Text"] = df_test["Text"].apply(lambda x: ' '.join(x))
df_test["Question"] = df_test["Question"].apply(lambda x: ' '.join(x))

In [ ]:
df_train

,Text,Question,Answer
0,Mary moved to the bathroom . Sandra journeyed ...,Is Sandra in the hallway ?,no
1,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bathroom ?,no
2,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the office ?,no
3,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes
4,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes
...,...,...,...
9995,Daniel went to the office . Sandra journeyed t...,Is Sandra in the kitchen ?,no
9996,Daniel went to the office . Sandra journeyed t...,Is Daniel in the bedroom ?,yes
9997,Daniel went to the office . Sandra journeyed t...,Is Sandra in the hallway ?,no
9998,Daniel went to the office . Sandra journeyed t...,Is Mary in the kitchen ?,no


In [ ]:
x_train = df_train[['Text', 'Question']].values  
y_train = df_train['Answer'].values 
x_test = df_test[['Text', 'Question']].values
y_test = df_test['Answer'].values

In [ ]:
x_train = list(map(tuple, x_train))
x_test = list(map(tuple, x_test))

In [ ]:
len(x_train)

10000

In [ ]:
type(y_train)

numpy.ndarray

In [ ]:
!pip install ktrain

In [ ]:
import ktrain # give string to ktrain 
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased' # needs to specify the name of the model
t = text.Transformer(MODEL_NAME, maxlen = 160, class_names=['no', 'yes']) # transformer API ( not sequential nor functional)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32) # lower bs if OOM occurs
learner.fit_onecycle(5e-5, 10)

preprocessing train...
language: en
sentence pairs detected


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/10
313/313 [==============================] - 165s 528ms/step - loss: 0.5031 - accuracy: 0.7277 - val_loss: 0.3798 - val_accuracy: 0.8280
Epoch 2/10
313/313 [==============================] - 164s 524ms/step - loss: 0.2929 - accuracy: 0.8765 - val_loss: 0.1061 - val_accuracy: 0.9610
Epoch 3/10
313/313 [==============================] - 164s 525ms/step - loss: 0.0691 - accuracy: 0.9764 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 4/10
313/313 [==============================] - 164s 525ms/step - loss: 0.0344 - accuracy: 0.9903 - val_loss: 0.0318 - val_accuracy: 0.9950
Epoch 5/10
313/313 [==============================] - 164s 525ms/step - loss: 0.0262 - accuracy: 0.9916 - val_loss: 0.0037 - val_accuracy: 0.9990
Epoch 6/10
313/313 [==============================] - 164s 524ms/step - loss: 0.0123 - accuracy: 0.9973 - val_loss: 9.1331e-04 - val_accuracy: 1.0000
Epoch 7/10
313/313 [==============================] - 164